##In this Notebook , 
A text Recogniser function is built for word and line level predictions .
The idea is to make it compatible with any text detector and added cer(character error rate),wer(word error rate),levenshtein distance as  performance metrics.

In [2]:
#required installations
!pip3 install --upgrade pip
!python3 -m pip install paddlepaddle -i https://mirror.baidu.com/pypi/simple

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 28.4 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 14.4 MB/s eta 0:00:00


In [4]:
#Installing requirements
!pip install -r /content/PaddleOCR/requirements.txt
!pip install python-Levenshtein
!pip install importlib-metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.0/948.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.2/604.2 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.5/204.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.8/404.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully 

In [5]:
import cv2
import os
import numpy as np
import sys
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import time
import numpy
 

In [6]:
%cd PaddleOCR/

/content/PaddleOCR


In [7]:
# Importing functions and methods for OCR
from tools.infer.predict_rec import *
import tools.infer.utility as utility
from ppocr.postprocess import build_post_process
from ppocr.utils.logging import get_logger
from ppocr.utils.utility import get_image_file_list, check_and_read_gif

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sc

In [26]:
#defining the function for recognition
def rec(args, out_path, input, rec_model_dir,rec_image_shape = "3, 32, 100", rec_char_type = "en", rec_algorithm = "CRNN", show = False, save = True):
    # Assigning values to args as the code is not running from the console.
    args.rec_model_dir = rec_model_dir
    args.rec_image_shape = rec_image_shape
    args.rec_char_type = rec_char_type
    args.rec_algorithm = rec_algorithm
    
 
    # Initializing some helper variables.
    t1 = 0
    t2 = 0
    tot = []
    os.chdir('/content/PaddleOCR')
 
    # Passing required values to the args variables.
    if rec_algorithm == "CRNN":
      args.rec_char_dict_path = './ppocr/utils/en_dict.txt'
      args.use_space_char = True

    if rec_algorithm == "SRN":
      args.rec_char_dict_path = './ppocr/utils/ic15_dict.txt'
      args.use_space_char = False
    
    if rec_algorithm == 'NRTR':
      args.rec_char_dict_path = './ppocr/utils/EN_symbol_dict.txt'
      args.rec_image_shape = "1,32,100"
    
    # Initializing recognizer.
    image_file_list = get_image_file_list(input)
    text_recognizer = TextRecognizer(args)
    valid_image_file_list = []
    img_list = []
 
    # Warming up the GPU to run it at its full capacity.
    if args.warmup:
        image = np.random.uniform(0, 255, [32, 320, 3]).astype(np.uint8)
        for i in range(10):
            res = text_recognizer([image])
 
    # Reading and appending all images' array to a list.
    for image_file in image_file_list:
        image, flag = check_and_read_gif(image_file)
        if not flag:
            image = cv2.imread(image_file)
        if image is None:
            logger.info("error in loading image:{}".format(image_file))
            continue
        valid_image_file_list.append(image_file)
        img_list.append(image)
 
    # Applying OCR to the images
    t1 = time.time()
    try:
        rec_res, _ = text_recognizer(img_list)
    except Exception as E:
        logger.info(traceback.format_exc())
        logger.info(E)
        exit()
    
    # Calculating FPS and printing the info.
    t2 = time.time()
    fps = str(t2-t1)
    for ino in range(len(img_list)):
        logger.info("Predicts of {}:{}".format(valid_image_file_list[ino],
                                              rec_res[ino]))
        if save:
          cv2.imwrite(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '_rec' + '.jpg'),img_list[ino])
          with open(os.path.join(out_path,valid_image_file_list[ino].split('/')[-1].split('.')[0] + '.txt'), 'w') as f:
            f.write(str(rec_res[ino]))
    logger.info("Time taken recognize all images : {}".format(fps))
    print(len(image_file_list))
    logger.info("Average fps : {}".format(1/(float(fps)/len(image_file_list))))
    
    #cv2.putText(image, str(rec_res[ino]),(0, int(15 * 1)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), thickness=1)
    
    # Displaying and saving the output according to the parameters set.
    #return valid_image_file_list,rec_res


In [11]:
#installations for metrics
!pip install pybind11
!pip install fastwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for fastwer: filename=fastwer-0.1.3-cp37-cp37m-linux_x86_64.whl size=638472 sha256=0e73f1878286ce81a8052f24356e955a1d384ab0bce2f6682e0886bd0b7c5241
  Stored in directory: /root/.cache/pip/wheels/c8/b6/93/419e758f0c0176d311602763520bcfdec18107be1f15186fe6
Successfully built fastwer


In [30]:
#defining the metric function which outputs cer,wer,levenshtein
#the function returns a dataframe which consists img,ocr,gt,cer,wer as its columns
def score_calc(pth, annot):
# Importing distance metric.
  from Levenshtein import distance
  import fastwer
  score_all = []
  # Create empty dataframe to store output
  df_output = pd.DataFrame(columns = ['img', 'ocr' , 'gt' , 'cer' , 'wer'])
  # Looping through the output text files and storing the OCR output in a variable.
  for out_file in os.listdir(pth):
    if out_file.endswith('.txt'):
      with open(os.path.join(pth, out_file), 'rb') as f:
        out = f.read()
        f.close()
      
      # Cleaning the OCR output text.
      try:
        out = str(out).split(',')[0].split(',')[0].replace("'", '').lower()
        #print(out[3:])
        out = out[3:]
        
      except:
        print('OCR output does not exist')
      #print(out_file.split('.')[0])
      # Opening the ground truth file and calculating weighted distance between ground truth and OCR output.
      with open(annot) as f:
        for line in f:
          #print(line.split('\t')[0].split('.')[0])
          if line.split('\t')[0].split('.')[0] == out_file.split('.')[0]:
            #print(out_file.split('.')[0], line.split('\t')[0])
            gt = line.split('\t')[1].lower()
            df_output = df_output.append({'img':str(out_file.split('.')[0]),'ocr':str(out),'gt':str(gt)},ignore_index=True)
            #print(df_output['img'])
            score = distance(str(out), str(gt))/len(gt)
            score_all.append(score)
            break
  # Printing the average score.
  #print(len(score_all))
  for index, row in df_output.iterrows():
    filename = row['img']
    ref = row['gt']
    output = row['ocr']
    cer = fastwer.score_sent(output, ref, char_level=True)
    wer = fastwer.score_sent(output, ref, char_level=False)
    df_output.loc[df_output['img'] == filename, 'cer'] = round(cer,2) # Round value to 2 decimal places
    df_output.loc[df_output['img'] == filename, 'wer'] = round(wer,2)
  print("final score:", sum(score_all)/len(score_all))
  # Overall performances
  mean_cer = df_output['cer'].mean()
  mean_wer = df_output['wer'].mean()
  #print(df_output.head())
  print('Mean CER = ',mean_cer,'%', 'Mean WER = ',mean_wer,'%')
  return df_output


In [14]:
os.mkdir('/content/rec_output')

In [28]:
# Running the OCR and storing the output in a folder specified under out_path in the root directory.

out_path = '/content/rec_check/'
rec_model_dir = '/content/PaddleOCR/inference/CRNN_mv3'
input = '/content/checking/check'
sys.argv = ['']
rec(utility.parse_args(), out_path, input, rec_model_dir, show = True)

[2022/07/08 08:02:28] ppocr INFO: Predicts of /content/checking/check/1411_0_cropped.jpg:('Medical Specialty.', 0.9822995662689209)
[2022/07/08 08:02:28] ppocr INFO: Predicts of /content/checking/check/1411_10_cropped.jpg:('cannot exclude the possibilty of recent stone passage although the findings are ultimately technically indeterminate and clinical correlation is', 0.9918806552886963)
[2022/07/08 08:02:28] ppocr INFO: Predicts of /content/checking/check/1411_11_cropped.jpg:('advised There is no obvious solid-appearing mass given the lack of contrast.', 0.9823830127716064)
[2022/07/08 08:02:28] ppocr INFO: Predicts of /content/checking/check/1411_12_cropped.jpg:('scans of the pelvis disclose no evidence of stone within the decompressed bladder No pelvic free fluid or adenopathy.', 0.9903239607810974)
[2022/07/08 08:02:28] ppocr INFO: Predicts of /content/checking/check/1411_13_cropped.jpg:('There are few scattered diverticula. There is a moderate amount of stoolthroughout the colon T

(['/content/checking/check/1411_0_cropped.jpg',
  '/content/checking/check/1411_10_cropped.jpg',
  '/content/checking/check/1411_11_cropped.jpg',
  '/content/checking/check/1411_12_cropped.jpg',
  '/content/checking/check/1411_13_cropped.jpg',
  '/content/checking/check/1411_14_cropped.jpg',
  '/content/checking/check/1411_15_cropped.jpg',
  '/content/checking/check/1411_16_cropped.jpg',
  '/content/checking/check/1411_17_cropped.jpg',
  '/content/checking/check/1411_18_cropped.jpg',
  '/content/checking/check/1411_19_cropped.jpg',
  '/content/checking/check/1411_1_cropped.jpg',
  '/content/checking/check/1411_20_cropped.jpg',
  '/content/checking/check/1411_21_cropped.jpg',
  '/content/checking/check/1411_22_cropped.jpg',
  '/content/checking/check/1411_23_cropped.jpg',
  '/content/checking/check/1411_24_cropped.jpg',
  '/content/checking/check/1411_2_cropped.jpg',
  '/content/checking/check/1411_3_cropped.jpg',
  '/content/checking/check/1411_4_cropped.jpg',
  '/content/checking/chec

In [31]:
result_path = '/content/rec_check/'
gt = '/content/checking/rec_check.txt'
df = score_calc(result_path, gt)


final score: 0.15254972573305706
Mean CER =  15.255026041666659 % Mean WER =  38.569270833333356 %


In [32]:
df.head()

,img,ocr,gt,cer,wer
0,2012_19_cropped,patient was sterilelypeepped and draped in the...,patient was sterilely prepped and draped in th...,5.26,18.52
1,1949_4_cropped,medical transcription sample reporty,(medical transcription sample report)\n,7.89,50.0
2,2658_30_cropped,diagnostic data: x-ray of the right ankle reve...,diagnostic data: x-ray of the right ankle reve...,1.6,11.11
3,1411_14_cropped,of acute diverticulitis. the appendix is normal.,of acute diverticulitis. the appendix is norma...,2.04,14.29
4,4942_27_cropped,electrocautery was used for hemostasis. a curv...,electrocautery was used for hemostasis. a curv...,2.78,13.04


Here the metric results are without alignments of strings